# Face Recognition against My Photo Library
The iCloud photos from iOS or Mac OS provided a comprenesive way to tag faces and then all the photos in the library will be scanned, faces will be identified and tagged accordingly based on the earlier user defined tags.

I am a photography enthusiast and I have started to taking photos from year 2004. As of today, it's over 200k shots and I've kept 40k of them. There are many desktop applications can do the job of face recognition, but it's going to be super fun if I can build the solution end to end.

This report will have following parts.
1. Get photos and preprocess.
2. Get the features of the baseine photos.
3. Train the model to perform the classification.
4. Evaluate the model.

## Get photos and preprocessing
### A glance at the folder structure
All my photos are in `D:\Pictures`, majority of them are in both `.jpg` and `.nef` format. The `.nef` is a raw image format for Nikon cameras and `.jpg` is the copy after image post-processing of raw file.

In [1]:
import os
cur_dir = os.getcwd()
print(cur_dir)
target_image_dir = os.path.join(cur_dir, 'images')
photo_dir = 'D:\Pictures'
os.listdir(photo_dir)

D:\Google Drive\Study\Deep Learning Developer\Projects\Project 4 - Face Recognition against My Photo Library


['.SynologyWorkingDirectory',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 'Adobe Lightroom',
 'Camera Roll',
 'desktop.ini',
 'iCloud Photos',
 'naming instruction.txt',
 'Phone Photos',
 'Photography Works',
 'Saved Pictures',
 'zbingjie',
 'zothers',
 '法蝶',
 '熊思宇和黄乐论辩论']

One sample of a recent photo directory. Each and every file use the time stamp as its file name. `.nef` is the raw image file, `.xmp` is generated by Adobe Lightroom, both are not applicable to this project.

In [2]:
os.listdir(photo_dir + '/2017/2017.11.16 - Singapore Fintech Festival')

['20171112-2038.jpg',
 '20171112-2038.NEF',
 '20171112-2038.xmp',
 '20171112-2039.jpg',
 '20171112-2039.NEF',
 '20171112-2039.xmp',
 '20171116-1705.jpg',
 '20171116-1705.NEF',
 '20171116-1705.xmp',
 '20171116-1707.jpg',
 '20171116-1707.NEF',
 '20171116-1707.xmp',
 '20171116-1708.jpg',
 '20171116-1708.NEF',
 '20171116-1708.xmp',
 '20171116-1709.jpg',
 '20171116-1709.NEF',
 '20171116-1709.xmp',
 '20171116-1710.jpg',
 '20171116-1710.NEF',
 '20171116-1710.xmp',
 '20171116-1713.jpg',
 '20171116-1713.NEF',
 '20171116-1713.xmp',
 '20171116-1715.jpg',
 '20171116-1715.NEF',
 '20171116-1715.xmp',
 '20171116-1716.jpg',
 '20171116-1716.NEF',
 '20171116-1716.xmp',
 '20171116-1717.jpg',
 '20171116-1717.NEF',
 '20171116-1717.xmp',
 '20171116-1718-2.jpg',
 '20171116-1718-2.NEF',
 '20171116-1718-2.xmp',
 '20171116-1718.jpg',
 '20171116-1718.NEF',
 '20171116-1718.xmp',
 '20171116-1720.jpg',
 '20171116-1720.NEF',
 '20171116-1720.xmp',
 '20171116-1721-2.jpg',
 '20171116-1721-2.NEF',
 '20171116-1721-2.xmp'

### Create face images
Iteratively going through all the images and save the face to a jpg file into the working directory. The value of the scaling factor in the cascade classifier affects the number of false positive. Some manual process will be needed.

In [3]:
# Import required libraries for this section
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import time

In [4]:
def show_faces(file_path):
    print('Image path', file_path)
    
    # The file path contains unicode characters, cannot use cv2.imread() directly
    file_stream = open(file_path, 'rb')
    bytes_arr = bytearray(file_stream.read())
    numpy_ar = np.asarray(bytes_arr, dtype=np.uint8)
    image = cv2.imdecode(numpy_ar, cv2.IMREAD_UNCHANGED)
    print(image.shape)
    
    # Convert to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert the RGB  image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Extract the pre-trained face detector from an xml file
    face_cascade = cv2.CascadeClassifier('detector_architectures/haarcascade_frontalface_default.xml')

    # Detect the faces in image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Print the number of faces detected in the image
    print('Number of faces detected:', len(faces))

    # Make a copy of the orginal image to draw face detections on
    image_with_detections = np.copy(image)

    # The list of detected faces
    image_faces = []
    # Get the bounding box for each detected face
    for (x,y,w,h) in faces:
        # Add a red bounding box to the detections image
        if w > 200:
            line_width = w//20
        else:
            line_width = 3
        image_faces.append(image[y:(y+h), x:(x+w)])
        cv2.rectangle(image_with_detections, (x,y), (x+w,y+h), (255,0,0), line_width)
    
    save_faces(file_path, image_faces)

    # Display the image with the detections
    #fig = plt.figure(figsize=(10, 10))
    #ax = fig.add_subplot(1, 1, 1, xticks=[], yticks=[])
    #ax.set_title('Sample Image')
    #ax.imshow(image_with_detections)
    os.chdir(cur_dir)

In [5]:
# pathlib available from python 3.5
from pathlib import Path
def save_faces(file_path, image_faces):
    if len(image_faces) == 0:
        return
    # Save the each face into individual files
    target_file = file_path.replace(photo_dir, target_image_dir)
    target_dir = os.path.dirname(target_file)
    target_path = Path(target_dir)
    
    # Create parents of directory, don't raise exception if the directory exists
    target_path.mkdir(parents=True, exist_ok=True)
        
    for i, face in enumerate(image_faces):
        face = cv2.resize(face, (299, 299))
        os.chdir(target_dir)
        file_name = os.path.basename(target_file)
        cv2.imwrite(file_name + '-face-' + str(i) + '.jpg', cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        

In [6]:
# Load in color image for face detection
file_path = os.path.join(photo_dir, '2017\\2017.11.16 - Singapore Fintech Festival', '20171116-1923.jpg')
show_faces(file_path)

Image path D:\Pictures\2017\2017.11.16 - Singapore Fintech Festival\20171116-1923.jpg
(4760, 7132, 3)
Number of faces detected: 5


In [7]:
all_jpg = []
for root, dirs, files in os.walk(photo_dir):
    # All the target photos are in D:\Pictures\20xx. Get the jpgs from them only.
    path = root.split(os.sep)
    if len(path) < 3:
        continue
    else:
        year = path[2]
        if year[:2] != '20':
            continue
    #print((len(path) - 1) * '---', os.path.basename(root))
    for file in files:
        if file[-3:].lower() == 'jpg':
            #print(len(path) * '---', file)
            all_jpg.append(os.path.join(root, file))

In [8]:
print('Number of jpgs:', len(all_jpg))
all_jpg[:10]

Number of jpgs: 39687


['D:\\Pictures\\2004\\2004.12.16 - 保存的第一张数码照片\\20041216-2037.JPG',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-0828.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-0829.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-1004.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-1011-2.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-1011.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-1013.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-1203.jpg',
 'D:\\Pictures\\2004\\2004.12.21~24 - 刚到新加坡，bukit timah hill, befriender, 猴子, oldham hall及附近\\20041221-1219.jpg',
 'D:\\Pictures\\200

In [ ]:
for i in range(len(all_jpg)):
    show_faces(all_jpg[i])